In [2]:
import praw
import os
from praw.models import MoreComments
import time
import pandas as pd
import spacy
from datetime import datetime, timezone

reddit = praw.Reddit(
    client_id=os.getenv('REDDIT_CLIENT_ID'),
    client_secret=os.getenv('REDDIT_SECRET'),
    username=os.getenv('REDDIT_USERNAME'),
    password=os.getenv('REDDIT_PASSWORD'),
    user_agent='crawler2/0.0.1'
)

query = 'OpenAI'





time_ranges = ["year", "month", "week", "day", "hour"]

# Set up spaCy for entity extraction
nlp = spacy.load("en_core_web_sm")

# Initialize a list to store the metadata for each post
metadata = []

# Search for the phrase in all subreddits within each time range
for time_range in time_ranges:
    for submission in reddit.subreddit("all").search(
        query, time_filter=time_range, sort="relevance", limit=None
    ):
        # Extract entities from the post title and body text
        doc = nlp(submission.title + " " + submission.selftext)
        entities = list(set([ent.text for ent in doc.ents]))

        # Convert the UTC timestamp to a datetime object
        utc_timestamp = datetime.fromtimestamp(submission.created_utc, timezone.utc)

        # Check if the URL is a Reddit URL or an external URL
        is_reddit_url = (
            submission.url.startswith("https://www.reddit.com")
            or submission.url.startswith("https://www.reddit.co")
        )

        # Get all comments on the post and store them in a dictionary
        comments_dict = {}
        submission.comments.replace_more(limit=None)
        for comment in submission.comments.list():
            # Extract entities from the comment body
            doc = nlp(comment.body)
            comment_entities = list(set([ent.text for ent in doc.ents]))

            # Create a dictionary containing the comment metadata and entities
            comment_dict = {
                "body": comment.body,
                "author": comment.author.name if comment.author else None,
                "score": comment.score,
                "created_utc": datetime.fromtimestamp(comment.created_utc, timezone.utc),
                "entities": comment_entities,
            }

            # Add the comment dictionary to the comments dictionary using the comment ID as the key
            comments_dict[str(comment.id)] = comment_dict

        # Append the metadata to the list
        metadata.append(
            {
                "title": submission.title,
                "url": submission.url,
                "is_reddit_url": is_reddit_url,
                "subreddit": submission.subreddit_name_prefixed,
                "author": submission.author.name if submission.author else None,
                "score": submission.score,
                "upvote_ratio": submission.upvote_ratio,
                "num_comments": submission.num_comments,
                "created_utc": utc_timestamp,
                "entities": entities,
                "comments": comments_dict,
            }
        )

# Convert the metadata list to a pandas DataFrame
df = pd.DataFrame(metadata)

# Save the DataFrame to a CSV file
df.to_csv("metadata.csv", index=False)

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df

,title,url,is_reddit_url,subreddit,author,score,upvote_ratio,num_comments,created_utc,entities,comments
0,"OpenAI CEO Says His Tech Is Poised to ""Break C...",https://futurism.com/the-byte/openai-ceo-agi-b...,False,r/Futurology,fungussa,24711,0.74,3987,2023-02-05 15:54:35+00:00,"[OpenAI, Break Capitalism""]",{'j7bn1af': {'body': 'The following submission...
1,"OpenAI stop moralizing us, we are not children.",https://i.redd.it/yzs4rb5tmhba1.png,False,r/ChatGPT,Unreal_777,444,0.85,307,2023-01-11 22:10:25+00:00,[OpenAI],{'j3yarxu': {'body': 'In order to prevent mult...
2,OpenAI pumps the breaks on GPT-4.,https://www.reddit.com/r/ChatGPT/comments/11cs...,True,r/ChatGPT,spiritus_dei,106,0.82,324,2023-02-26 21:04:15+00:00,"[#, Sam Altman's, Russia, Microsoft, China, Go...",{'ja4rztv': {'body': 'To avoid redundancy in t...
3,OpenAI Is Now Everything It Promised Not to Be...,https://www.vice.com/en/article/5d3naz/openai-...,False,r/technology,speckz,8681,0.97,827,2023-02-28 22:17:31+00:00,"[OpenAI, For-Profit]",{'jaf79jn': {'body': '10 billion dollars is a ...
4,"OpenAI is dumbing down ChatGPT, again",https://www.reddit.com/r/OpenAI/comments/zwomk...,True,r/OpenAI,Mk_Makanaki,211,0.81,152,2022-12-27 20:03:48+00:00,"[#, AI, OpenAI, less than a month, Keep, About...",{'j1w5ua6': {'body': 'I'd like to point out th...
...,...,...,...,...,...,...,...,...,...,...,...
987,Ai MC Ride,https://i.redd.it/63dlpcrp22na1.jpg,False,r/deathgrips,Snoo_28756,2,0.75,1,2023-03-11 01:40:07+00:00,[],{'jbr4mj2': {'body': 'mc ride if he was a good...
988,What the hell? I used one o' them 'glitch toke...,https://www.reddit.com/gallery/11o7hzq,True,r/ChatGPT,RandoEncounter,1,1.00,1,2023-03-11 01:48:06+00:00,[],{'jbr5ifi': {'body': 'To avoid redundancy of s...
989,A hit is a hit.....,https://i.redd.it/i0h7443ym0na1.jpg,False,r/CirclejerkSopranos,PeterMarieAndSandy,2,1.00,1,2023-03-11 01:50:24+00:00,[],{'jbr5v62': {'body': 'Christopher Moltisanti s...
990,Request: Can somebody test the experimental mo...,https://www.reddit.com/r/dalle2/comments/11o7c...,True,r/dalle2,BeginningInfluence55,1,1.00,1,2023-03-11 01:40:42+00:00,[],{'jbr4l3j': {'body': '**Welcome to** r/dalle2!...


In [2]:
import praw
import os
from praw.models import MoreComments
import time

reddit = praw.Reddit(
    client_id=os.getenv('REDDIT_CLIENT_ID'),
    client_secret=os.getenv('REDDIT_SECRET'),
    username=os.getenv('REDDIT_USERNAME'),
    password=os.getenv('REDDIT_PASSWORD'),
    user_agent='crawler2/0.0.1'
)

query = 'OpenAI'

# Define time ranges for search
time_ranges = ['year', 'month', 'week', 'day', 'hour']

for time_range in time_ranges:
    # Search for the phrase in all subreddits within the current time range
    submissions = reddit.subreddit('all').search(query, sort='relevance', syntax='lucene', time_filter=time_range)

    for submission in submissions:
        print(submission.title)
        print(submission.url)
        print(submission.subreddit.display_name)

    # Wait for 2 seconds before searching again (to avoid hitting the rate limit)
    time.sleep(2)

OpenAI CEO Says His Tech Is Poised to "Break Capitalism"
https://futurism.com/the-byte/openai-ceo-agi-break-capitalism
Futurology
OpenAI stop moralizing us, we are not children.
https://i.redd.it/yzs4rb5tmhba1.png
ChatGPT
OpenAI pumps the breaks on GPT-4.
https://www.reddit.com/r/ChatGPT/comments/11csn7d/openai_pumps_the_breaks_on_gpt4/
ChatGPT
OpenAI Is Now Everything It Promised Not to Be: Corporate, Closed-Source, and For-Profit
https://www.vice.com/en/article/5d3naz/openai-is-now-everything-it-promised-not-to-be-corporate-closed-source-and-for-profit
technology
OpenAI is dumbing down ChatGPT, again
https://www.reddit.com/r/OpenAI/comments/zwomkm/openai_is_dumbing_down_chatgpt_again/
OpenAI
Meet OpenAI CEO Sam Altman, who learned to code at 8 and is a doomsday prepper with a stash of gold, guns, and gas masks
https://businessinsider.com/sam-altman-chatgpt-openai-ceo-career-net-worth-ycombinator-prepper-2023-1
technology
OpenAI has hired an army of contractors to make basic coding ob

In [1]:
import praw
import os
from praw.models import MoreComments

# Initialize PRAW with your Reddit API credentials
reddit = praw.Reddit(
    client_id=os.getenv('REDDIT_CLIENT_ID'),
    client_secret=os.getenv('REDDIT_SECRET'),
    username=os.getenv('REDDIT_USERNAME'),
    password=os.getenv('REDDIT_PASSWORD'),
    user_agent='crawler2/0.0.1'
)



# Define the keyword or phrase to search for
keyword = 'OpenAI'

# Search for all posts that contain the keyword
posts = reddit.subreddit('all').search(keyword, limit=None, time_filter='all', sort='new', syntax='cloudsearch')

# Create a new text file to save the posts
filename = f'{keyword}_posts.txt'
with open(filename, 'w', encoding='utf-8') as file:
    
    # Loop through each post and write the title and URL to the file
    for post in posts:
        file.write(f'Title: {post.title}\n')
        file.write(f'URL: {post.url}\n\n')

9eBLZxwGodkz5W2LCv39ow
VpF7liJL4N3OChpwHoG3bTSBi1bh2g
Apprehensive-Cup6910
Pierogies5$


In [5]:
temp = reddit.subreddit('all').search(keyword, limit=None, time_filter='all', sort='new', syntax='cloudsearch')

In [6]:
for i in temp:
    print(i.title)
    print(i.url)
    print(i.selftext)
    print(i.comments)
    print(i.subreddit)
    print(i.score)
    print(i.id)
    print(i.created)
    break

ResponseException: received 401 HTTP response

In [26]:
import praw

def get_data_reddit(search, after=None):
    r = reddit
    params = {"q": search}
    if after:
        params["after"] = "t3_" + str(after.id)
    posts = r.get_content(r.config['search'] % 'all', params=params, limit=100)
    return posts

search = "Elon Musk"
post = None
count = 0
while True:
    posts = get_data_reddit(search, post)
    for post in posts:
        print(str(post.id))
        count += 1
    print(count)

AttributeError: 'Reddit' object has no attribute 'get_content'